# Import librairie #

In [1]:
import psycopg2
import pandas as pd
from dotenv import load_dotenv
import os

## Charger les variables d'environnement ##

In [2]:
# Charger le .env depuis la racine du projet
load_result = load_dotenv(dotenv_path='../../.env')
print(f"Chargement .env : {'✓ Réussi' if load_result else '✗ Échec'}")

Chargement .env : ✓ Réussi


## Connexion Redshift Pennylane ##

In [3]:
conn = psycopg2.connect(
    host='pennylane-external.csqwamh5pldr.eu-west-1.redshift.amazonaws.com',
    port=5439,
    dbname='prod',
    user='u_289572',
    password=os.getenv('PENNYLANE_DATA_SHARING_KEY')
)

print("Connexion Redshift etablie")

Connexion Redshift etablie


## Fonction helper : Connexion auto-reconnect ##

In [4]:
def get_active_connection():
    """Retourne une connexion active, la recrée si nécessaire"""
    global conn
    try:
        # Test si la connexion est active
        pd.read_sql("SELECT 1;", conn)
        return conn
    except:
        # Reconnexion si fermée
        print("⟳ Reconnexion Redshift...")
        conn = psycopg2.connect(
            host='pennylane-external.csqwamh5pldr.eu-west-1.redshift.amazonaws.com',
            port=5439,
            dbname='prod',
            user='u_289572',
            password=os.getenv('PENNYLANE_DATA_SHARING_KEY')
        )
        print("✓ Reconnecté")
        return conn

# Utilisation : remplace "conn" par "get_active_connection()" dans tes requêtes
# Exemple :
df = pd.read_sql("SELECT * FROM pennylane.customers LIMIT 1;", get_active_connection())
print(f"Test réussi : {len(df)} ligne(s)")

C:\Users\ycloa\AppData\Local\Temp\ipykernel_27468\3220976796.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT 1;", conn)
C:\Users\ycloa\AppData\Local\Temp\ipykernel_27468\3220976796.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM pennylane.customers LIMIT 1;", get_active_connection())


Test réussi : 1 ligne(s)


## Liste table declaration TVA ##

In [5]:
# Tables tax declarations Pennylane
tables_extended = [
    'tax_declarations'
]

accessible_tables = []

for table in tables_extended:
    try:
        query = f"SELECT * FROM pennylane.{table} LIMIT 1;"
        df = pd.read_sql(query, conn)
        accessible_tables.append({
            'table_name': table,
            'columns': len(df.columns),
            'column_list': ', '.join(df.columns.tolist()[:5]) + '...'
        })
        print(f"✓ {table} ({len(df.columns)} colonnes)")
    except:
        pass

print(f"\n\n=== RÉSUMÉ: {len(accessible_tables)} tables accessibles ===")
df_accessible = pd.DataFrame(accessible_tables)
print(df_accessible.to_string(index=False))

C:\Users\ycloa\AppData\Local\Temp\ipykernel_27468\523721459.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✓ tax_declarations (15 colonnes)


=== RÉSUMÉ: 1 tables accessibles ===
      table_name  columns                                             column_list
tax_declarations       15 id, company_id, company_name, declaration_type, form...


## Liste declaration TVA Pennylane ##

In [6]:
# Afficher les colonnes tax declarations
tables_found = [
    'tax_declarations'
]

for table in tables_found:
    query = f"SELECT * FROM pennylane.{table} LIMIT 0;"  # LIMIT 0 = structure seulement
    df = pd.read_sql(query, conn)
    print(f"\n{'='*60}")
    print(f"Table: pennylane.{table}")
    print(f"{'='*60}")
    print(f"Colonnes ({len(df.columns)}):")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i:2d}. {col}")


Table: pennylane.tax_declarations
Colonnes (15):
   1. id
   2. company_id
   3. company_name
   4. declaration_type
   5. form
   6. declaration_id
   7. declaration_period_start_at
   8. declaration_period_end_at
   9. declaration_status
  10. is_declaration_validated
  11. is_payment_validated
  12. payment_amount
  13. reason_for_rejection
  14. updated_at
  15. created_at


C:\Users\ycloa\AppData\Local\Temp\ipykernel_27468\1012273436.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


## Chargement des données declaration TVA ##

In [7]:
# Charger les declaration TVA
query_gl = "SELECT * FROM pennylane.tax_declarations;"
df_tax_declarations = pd.read_sql(query_gl, get_active_connection())

print(f"✓ Données chargées : {len(df_tax_declarations)} lignes, {len(df_tax_declarations.columns)} colonnes")

C:\Users\ycloa\AppData\Local\Temp\ipykernel_27468\3220976796.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT 1;", conn)
C:\Users\ycloa\AppData\Local\Temp\ipykernel_27468\122018122.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tax_declarations = pd.read_sql(query_gl, get_active_connection())


✓ Données chargées : 18 lignes, 15 colonnes


## Affichage des colonnes disponibles ##

In [8]:
print("Colonnes disponibles :")
for i, col in enumerate(df_tax_declarations, 1):
    print(f"  {i:2d}. {col}")

Colonnes disponibles :
   1. id
   2. company_id
   3. company_name
   4. declaration_type
   5. form
   6. declaration_id
   7. declaration_period_start_at
   8. declaration_period_end_at
   9. declaration_status
  10. is_declaration_validated
  11. is_payment_validated
  12. payment_amount
  13. reason_for_rejection
  14. updated_at
  15. created_at


## Affichage type colonne ##

In [9]:
print(f"Types de données des {len(df_tax_declarations.columns)} colonnes :\n")
for i, (col, dtype) in enumerate(df_tax_declarations.dtypes.items(), 1):
    print(f"  {i:2d}. {col:<30} → {dtype}")

Types de données des 15 colonnes :

   1. id                             → int64
   2. company_id                     → int64
   3. company_name                   → object
   4. declaration_type               → object
   5. form                           → object
   6. declaration_id                 → float64
   7. declaration_period_start_at    → object
   8. declaration_period_end_at      → object
   9. declaration_status             → object
  10. is_declaration_validated       → object
  11. is_payment_validated           → object
  12. payment_amount                 → float64
  13. reason_for_rejection           → object
  14. updated_at                     → datetime64[ns]
  15. created_at                     → datetime64[ns]


## Conversion des types de colonnes ##

In [10]:
# Conversion des types
df_tax_declarations = df_tax_declarations.astype({
    'updated_at':'datetime64[ns]',
    'payment_amount':'float64',
    'created_at':'datetime64[ns]'
})

# Convertir toutes les autres colonnes en string
columns_to_string = [col for col in df_tax_declarations.columns if col not in ['updated_at','payment_amount','created_at']]
df_tax_declarations[columns_to_string] = df_tax_declarations[columns_to_string].astype(str)

print("✓ Conversion effectuée\n")
print("Nouveaux types :")
for i, (col, dtype) in enumerate(df_tax_declarations.dtypes.items(), 1):
    print(f"  {i:2d}. {col:<30} → {dtype}")

✓ Conversion effectuée

Nouveaux types :
   1. id                             → object
   2. company_id                     → object
   3. company_name                   → object
   4. declaration_type               → object
   5. form                           → object
   6. declaration_id                 → object
   7. declaration_period_start_at    → object
   8. declaration_period_end_at      → object
   9. declaration_status             → object
  10. is_declaration_validated       → object
  11. is_payment_validated           → object
  12. payment_amount                 → float64
  13. reason_for_rejection           → object
  14. updated_at                     → datetime64[ns]
  15. created_at                     → datetime64[ns]


## Connexion PostgreSQL local ##

In [11]:
# Connexion à PostgreSQL local (utilise les variables du .env)
conn_pg = psycopg2.connect(
    host=os.getenv('POSTGRES_HOST'),
    port=int(os.getenv('POSTGRES_PORT')),
    dbname=os.getenv('POSTGRES_DB'),
    user=os.getenv('POSTGRES_USER'),
    password=os.getenv('POSTGRES_PASSWORD')
)

print("✓ Connexion PostgreSQL établie")
print(f"  Database: {os.getenv('POSTGRES_DB')}")
print(f"  User: {os.getenv('POSTGRES_USER')}")
print(f"  Port: {os.getenv('POSTGRES_PORT')}")

✓ Connexion PostgreSQL établie
  Database: pennylane_data
  User: pennylane_user
  Port: 5433


## Import des données dans PostgreSQL avec SQLAlchemy ##

In [12]:
from sqlalchemy import create_engine

# Créer un engine SQLAlchemy
engine = create_engine(
    f"postgresql://{os.getenv('POSTGRES_USER')}:{os.getenv('POSTGRES_PASSWORD')}@{os.getenv('POSTGRES_HOST')}:{os.getenv('POSTGRES_PORT')}/{os.getenv('POSTGRES_DB')}"
)

print("✓ Engine SQLAlchemy créé")

✓ Engine SQLAlchemy créé


## Export du DataFrame vers PostgreSQL ##